## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [ ]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import string
from string import punctuation
import re
import os
import pandas as pd
import numpy as np
import os
from nltk.corpus import stopwords

stops = stopwords.words('russian')

In [3]:
from pymystem3 import Mystem

mystem = Mystem()

In [4]:
def cleaner(text):
    new_text = re.sub('\ufeff', '', text)
    new_text = re.sub('([\W]|\d)', ' ', new_text)
    new_text = re.sub('(\n|\t|\v|\r)', ' ', new_text)
    
    return new_text

In [5]:
main_dir = '/Users/alinashaymardanova/Downloads/Friends'

i = 0
freq = list()
files_list = {}
episod_name = {}

for root, dirs, files in os.walk(main_dir):
    for name in files:
        if name != '.DS_Store':
            with open(os.path.join(root, name), 'r') as f:
                name = re.sub('\.ru\.txt', '', name)
                text = f.read()
                lemma = mystem.lemmatize(cleaner(text))
                freq.append(len(lemma))
                files_list[name] = ''.join(lemma)
                episod_name[i] = name
                i += 1


### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов

In [6]:
len(files_list)

165

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [8]:
vect = TfidfVectorizer()
x = vect.fit_transform(list(files_list.values()))

In [9]:
data = pd.DataFrame(x.todense()).transpose()
data.columns = list(episod_name.values())

data['Word'] = vect.get_feature_names()
data = data.set_index(['Word'])
data['freq'] = data.sum(axis=1)

In [10]:
data.head()

,Friends - 2x08 - The One With The List,Friends - 2x20 - The One Where Old Yeller Dies.NurlanB,Friends - 2x24 - The One With Barry And Mindy's Wedding,Friends - 2x16 - The One Where Joey Moves Out,Friends - 2x07 - The One Where Ross Finds Out,Friends - 2x17 - The One Where Eddie Moves In,Friends - 2x04 - The One With Phoebe's Husband,Friends - 2x15 - The One Where Ross And Rachel...You Know,Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip,Friends - 2x21 - The One With The Bullies,...,Friends - 7x08 - The One Where Chandler Doesn't Like Dogs,Friends - 7x09 - The One With All The Candy,Friends - 7x17 - The One With The Cheap Wedding Dress,Friends - 7x23 - The One With Chandler And Monica's Wedding (1),Friends - 7x13 - The One Where Rosita Dies,Friends - 7x11 - The One With All The Cheesecakes,Friends - 7x12 - The One Where They're Up All Night,Friends - 7x03 - The One With Phoebe's Cookies,Friends - 7x01 - The One With Monica's Thunder,freq
Word,,,,,,,,,,,,,,,,,,,,,
after,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.025743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025743
again,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025580
ahh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024607
all,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068348
and,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.076152


In [11]:
def searcher(data, word):
    
    try:
        new_df = pd.DataFrame(data.transpose()[str(word)])
        result = new_df.loc[new_df[str(word)] != 0].transpose().columns.values[:-1]
    
    except KeyError:
        raise ValueError('No such word')

    return list(result)


In [12]:
searcher(data, 'all')

['Friends - 5x18 - The One Where Rachel Smokes',
 'Friends - 6x01 - The One After Vegas']

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [13]:
from collections import defaultdict

In [14]:
def inverted_index(vect) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    
    dic = defaultdict(list)

    for word in vect.get_feature_names():
        list_ = list()
        for el in searcher(data, word):
            list_.append(el)
        dic[word] = list_
    
    return dic

In [15]:
dic = inverted_index(vect)

In [16]:
visual_df = pd.DataFrame()
visual_df['word'] = dic.keys()
visual_df['episod'] = dic.values()
visual_df.head()

,word,episod
0,after,[Friends - 7x09 - The One With All The Candy]
1,again,[Friends - 5x19 - The One Where Ross Can't Flirt]
2,ahh,[Friends - 5x02 - The One With All The Kissing]
3,all,"[Friends - 5x18 - The One Where Rachel Smokes,..."
4,and,[Friends - 4x01 - The One With The Jellyfish.T...


С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


In [17]:
import collections

In [18]:
df_inv = pd.DataFrame({'word': vect.get_feature_names(),
                        'episod': list(dic.values()),
                        'freq': data['freq']})

df_inv['count'] = df_inv.episod.map(lambda x: len(x))
# df_inv = df_inv.drop('episod', 1)

In [19]:
df_inv.shape

(14115, 4)

+ Какое слово является самым частотным?

In [20]:
df_inv.sort_values('freq', ascending=False).head(1)

,episod,freq,word,count
Word,,,,
ты,"[Friends - 2x08 - The One With The List, Frien...",55.330057,ты,165


+ Какое самым редким?

In [21]:
df_inv.sort_values('freq').head(n=1)

,episod,freq,word,count
Word,,,,
туннельный,[Friends - 4x23-24 - The One With Ross's Weddi...,0.015636,туннельный,1


+ Какой набор слов есть во всех документах коллекции?

In [22]:
list(df_inv[df_inv['count'] == 165]['word'])

['быть',
 'весь',
 'все',
 'да',
 'давать',
 'думать',
 'если',
 'еще',
 'знать',
 'как',
 'мой',
 'мочь',
 'мы',
 'на',
 'не',
 'нет',
 'но',
 'ну',
 'он',
 'она',
 'они',
 'просто',
 'сказать',
 'так',
 'такой',
 'то',
 'ты',
 'хорошо',
 'хотеть',
 'что',
 'это',
 'этот']

+ Какой сезон был самым популярным у Чендлера?

In [23]:
def counter(name):
    seasons = dict()
    out = df_inv[df_inv['word'] == name]
    for episod in out['episod']:
        for el in episod:
            num = re.search('Friends - ([0-9]+)?x', str(el)).group(1)
            if num in seasons:
                seasons[num] += 1
            else:
                seasons[num] = 0
                seasons[num] += 1
    
    return seasons

In [24]:
from operator import itemgetter

In [25]:
result = counter('чендлер')

sorted(result.items(), key=itemgetter(1))[-1][0]

'6'

+ Какой сезон был самым популярным у Моники?

In [26]:
result = counter('моника')

sorted(result.items(), key=itemgetter(1))[-1][0]

'2'

+ Кто из главных героев статистически самый популярный?

In [27]:
data.loc[['моника', 'чендлер', 'росс', 'джоуи', 'фиби', 'рэйчел']]['freq'].sort_values()

Word
рэйчел     2.427839
фиби       3.236068
моника     3.881567
джоуи      3.999845
чендлер    4.084930
росс       5.182266
Name: freq, dtype: float64

## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [28]:
from math import log

In [33]:
k1 = 2.0
b = 0.75
N = 165
n = 0
avgdl = np.mean(freq)

In [34]:
def score_BM25(input_, episod, avgdl, k1, b, N, n, data):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    
    """
    arr = mystem.lemmatize(cleaner(input_))
    result = 0 
    for word in arr:
        if (word in data.index) and (episod in data.columns):
            n = df_inv.loc[word]['count'] 
            f = data.loc[word][episod] / (log((N - n + 0.5) / n + 0.5))
            a = (k1 + 1) * f
            b = f + k1 * (1 - b + b * (len(arr)) / avgdl)
            result += (a / b) * log((N - n + 0.5) / n + 0.5)
    
    return result

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [37]:
def compute_sim(text, doc, data) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    result = score_BM25(text, doc, avgdl, k1, b, N, n, data)
    
    return result


def get_search_result(text, data) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    res = []
    
    for doc in data.columns:
        res.append((doc, compute_sim(text, doc, data)))
    
    return res

In [38]:
sorted(get_search_result('рождественские каникулы', data), key=lambda x: x[1], reverse=True)[1:11]

[('Friends - 7x10 - The One With The Holiday Armadillo', 0.67619165266019921),
 ('Friends - 3x10 - The One Where Rachel Quits', 0.49429912695967915),
 ("Friends - 6x19 - The One With Joey's Fridge", 0.35909711476235001),
 ("Friends - 2x09 - The One With Phoebe's Dad", 0.32930672640481667),
 ('Friends - 1x17 - The One With Two Parts (2)', 0.23395328545092364),
 ('Friends - 1x16 - The One With Two Parts (1)', 0.2240512194498003),
 ("Friends - 4x03 - The One With The 'Cuffs", 0.21922985809068396),
 ('Friends - 4x10 - The One With The Girl From Poughkeepsie',
  0.20922793379975604),
 ('Friends - 6x10 - The One With The Routine', 0.11490356515806868),
 ('Friends - 4x08 - The One With Chandler In A Box', 0.1107416464352531)]